# PREDICT DATA

In [1]:
#r "nuget:Microsoft.ML,1.5.2"
using Microsoft.ML;
using Microsoft.ML.Data;

Installed package Microsoft.ML version 1.5.2

### Load data models from Models.cs file 

In [1]:
#load "C:\Users\dcost\source\repos\SmartFireAlarm\SmartFireAlarm\Jupyter\Models.csx" 

### Initialize the ML context (we need it for building the data and training pipelines)  

In [1]:
MLContext mlContext = new MLContext(seed: 123);

In [1]:
const string DATASET_PATH = "./sensors_data.csv";
IDataView data = mlContext.Data.LoadFromTextFile<ModelInput>(
    path: DATASET_PATH,
    hasHeader: true,
    separatorChar: ',');

In [1]:
var shuffledData = mlContext.Data.ShuffleRows(data, seed: 1);
var split = mlContext.Data.TrainTestSplit(shuffledData, testFraction: 0.2);
var trainingData = split.TrainSet;
var testingData = split.TestSet;

In [1]:
var featureColumns = new[] { "Temperature", "Luminosity", "Infrared", "Distance", "Hour", "Day" };

var trainingPipeline = mlContext.Transforms.Conversion.MapValueToKey("Label")
    .Append(mlContext.Transforms.CustomMapping<CustomInputRow, CustomOutputRow>
        (CustomMappings.IncomeMapping, nameof(CustomMappings.IncomeMapping)))
    .Append(mlContext.Transforms.Concatenate("Features", featureColumns))
    .Append(mlContext.Transforms.NormalizeMinMax("Features"))
    .Append(mlContext.MulticlassClassification.Trainers.SdcaNonCalibrated("Label", "Features"))
    .Append(mlContext.Transforms.Conversion.MapKeyToValue("PredictedLabel"));

### Train the model 

In [1]:
var model = trainingPipeline.Fit(trainingData);

### Evaluate the model 

In [1]:
var predictions = model.Transform(testingData);
var metrics = mlContext.MulticlassClassification.Evaluate(predictions, "Label", "Score", "PredictedLabel");
display(p[@style:"font-size: 40px"]($"MicroAccuracy: {metrics.MicroAccuracy:P2}, MacroAccuracy: {metrics.MacroAccuracy:P2}"));

MicroAccuracy: 98.20%, MacroAccuracy: 97.41%

### Generate a sample input

In [1]:
var sampleData = new ModelInput
{
    Luminosity = 92F,
    Temperature = 22F,
    Infrared = 0F,
    Distance = 0F,
    CreatedAt = "01/03/2020 10:22:08"
};

## Predict data

In [1]:
#r "nuget:MathNet.Numerics"
#load "C:\Users\dcost\source\repos\SmartFireAlarm\SmartFireAlarm\Jupyter\Helpers.csx"

Installed package MathNet.Numerics version 4.12.0

In [1]:
var predictor = mlContext.Model.CreatePredictionEngine<ModelInput, ModelOutput>(model);
var predicted = predictor.Predict(sampleData);

display(p[@style:"font-size: 40px"]($"{predicted.PredictedLabel}"));

display(Helpers.GetPredictionPerClass(predicted, predictor.OutputSchema));

FlashLight

5.8467,FlashLight
-0.1722,Lighter
-1.2180,Infrared
-4.1925,Day
